In [1]:
import pymongo
import pandas as pd 
import re
from sklearn.feature_extraction.text import TfidfVectorizer

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["story_database"]  # Database name
collection = db["short_stories"]

documents = collection.find()

df_raw = pd.DataFrame(list(documents))
df_raw = df_raw[["text", "label"]]
df_raw.head(5)

,text,label
0,Transcriber's Note:\nEvery effort has been mad...,0
1,"“In a few moments Marianne, Solomin, Paul,\n\n...",0
2,“Marianne knelt beside the sofa.… Nezhdanof\n\...,0
3,"“Solomin raised Marianne's hand, her head\n\nl...",0
4,"“He now was no longer, but the hands of\n\nSol...",0


In [2]:
# Define the cleaning function
def clean_text(text):

    text = text.replace('\n', ' ') 
    text = re.sub(r'=', '', text)  

    # Keep only English alphabet characters and spaces 
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove extra whitespace
    text = ' '.join(text.split())

    return text

df_raw['cleaned_text'] = df_raw['text'].apply(clean_text)

df = df_raw[["cleaned_text", "label"]]

# Vectorize the data


In [3]:
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

df_shuffled = df.sample(frac=1).reset_index(drop=True)


X = df_shuffled["cleaned_text"]
y = df_shuffled["label"]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


clf = LogisticRegression(max_iter=1000)  # max_iter to ensure convergence
clf.fit(X_train, y_train)
        
y_pred = clf.predict(X_test)

cleaned_data = (classification_report(y_test, y_pred))
print(cleaned_data)

              precision    recall  f1-score   support

           0       1.00      0.68      0.81        37
           1       0.81      1.00      0.89        50

    accuracy                           0.86        87
   macro avg       0.90      0.84      0.85        87
weighted avg       0.89      0.86      0.86        87



Wow this was a bit anticlimactic, lets mess the data up see if we get the same results

# Lets not clean the data!

In [4]:
df_shuffled = df_raw.sample(frac=1).reset_index(drop=True)

X = df_shuffled["cleaned_text"]
y = df_shuffled["label"]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


clf = LogisticRegression(max_iter=1000)  # max_iter to ensure convergence
clf.fit(X_train, y_train)
        
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.97      0.81      0.88        37
           1       0.88      0.98      0.92        50

    accuracy                           0.91        87
   macro avg       0.92      0.90      0.90        87
weighted avg       0.91      0.91      0.91        87



# Lets add data, fakenews!

In [5]:
fakenews = pd.read_csv("data/fake_news.csv")
fakenews.columns = ["cleaned_text", "label"]

In [6]:
data = pd.concat([fakenews, df], ignore_index=True)


In [7]:
df_shuffled = data.sample(frac=1).reset_index(drop=True)

X = df_shuffled["cleaned_text"]
y = df_shuffled["label"]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


clf = LogisticRegression(max_iter=1000)  # max_iter to ensure convergence
clf.fit(X_train, y_train)
        
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      0.76      0.86        42
           1       0.81      0.98      0.89        49
           2       0.89      0.89      0.89        46

    accuracy                           0.88       137
   macro avg       0.90      0.88      0.88       137
weighted avg       0.90      0.88      0.88       137



In [8]:
confusion_matrix(y_test, y_pred)

array([[32,  6,  4],
       [ 0, 48,  1],
       [ 0,  5, 41]], dtype=int64)

## Is the vectorizer that good?

In [9]:
bbc_news = pd.read_csv("data/bbc_news.csv")
bbc = bbc_news[["cleaned_text", "category_encoded"]]
bbc.columns =  ["cleaned_text", "label"]

In [10]:
data = pd.concat([bbc, data], ignore_index=True)

In [11]:
data["label"].value_counts()

label
3    100
6    100
5    100
7    100
4    100
2    100
1    100
0     74
Name: count, dtype: int64

In [12]:
df_shuffled = data.sample(frac=1).reset_index(drop=True)

X = df_shuffled["cleaned_text"]
y = df_shuffled["label"]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


clf = LogisticRegression(max_iter=1000)  # max_iter to ensure convergence
clf.fit(X_train, y_train)
        
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.86      0.92        21
           1       0.59      0.90      0.72        21
           2       0.83      0.69      0.75        29
           3       0.97      0.93      0.95        30
           4       0.91      0.91      0.91        34
           5       0.86      0.94      0.90        33
           6       0.96      0.90      0.93        29
           7       1.00      0.92      0.96        36

    accuracy                           0.88       233
   macro avg       0.89      0.88      0.88       233
weighted avg       0.90      0.88      0.89       233



In [13]:
confusion_matrix(y_test, y_pred)

array([[18,  1,  2,  0,  0,  0,  0,  0],
       [ 0, 19,  1,  0,  0,  0,  1,  0],
       [ 0,  4, 20,  0,  3,  2,  0,  0],
       [ 0,  1,  0, 28,  0,  1,  0,  0],
       [ 0,  3,  0,  0, 31,  0,  0,  0],
       [ 0,  1,  0,  1,  0, 31,  0,  0],
       [ 0,  0,  1,  0,  0,  2, 26,  0],
       [ 0,  3,  0,  0,  0,  0,  0, 33]], dtype=int64)

## Increasing the datas dimensions by adding more data as well as adding more things to classify is instead of worsening the model its getting improved?
The TfidfVectorizer proves to be an excellent tool for vectorizing text data, as it does a near-perfect job of capturing the essence of the information. However, it's important to note that a vectorizer like TfidfVectorizer thrives on large datasets. While adding more labels was part of the improvement, it was the accompanying increase in data that enabled the creation of a more complex sparse matrix, allowing the model to perform more efficiently and effectively.

## Lets create a transformer

In [ ]:
df_deeplearning = df_shuffled

In [19]:
from transformers import AutoTokenizer

# Load a tokenizer — you can use any pretrained transformer model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Use tokenizer to encode the cleaned_text column
df_deeplearning['input_ids'] = df_deeplearning['cleaned_text'].apply(
    lambda x: tokenizer.encode(x, truncation=True, padding='max_length', max_length=128)
)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\felik\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\felik\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [21]:
df_deeplearning

,cleaned_text,label,token,input_ids
0,GE sees excellent world economy US behemoth G...,3,"[GE, sees, excellent, world, economy, US, behe...","[101, 16216, 5927, 6581, 2088, 4610, 2149, 202..."
1,Stuart joins Norwich from Addicks Norwich hav...,7,"[Stuart, joins, Norwich, from, Addicks, Norwic...","[101, 6990, 9794, 12634, 2013, 5587, 6799, 201..."
2,Movie body hits peer-to-peer nets The movie i...,4,"[Movie, body, hits, peer-to-peer, nets, The, m...","[101, 3185, 2303, 4978, 8152, 1011, 2000, 1011..."
3,Arnesen denies rift with Santini Tottenham sp...,7,"[Arnesen, denies, rift, with, Santini, Tottenh...","[101, 12098, 14183, 2078, 23439, 16931, 2007, ..."
4,Who do you think you are? The real danger is ...,4,"[Who, do, you, think, you, are?, The, real, da...","[101, 2040, 2079, 2017, 2228, 2017, 2024, 1029..."
...,...,...,...,...
769,"“…In 5 cities 8 kings; they ruled for 241,200 ...",2,"[“…In, 5, cities, 8, kings;, they, ruled, for,...","[101, 1523, 1529, 1999, 1019, 3655, 1022, 5465..."
770,"Analysis: No pain, no gain? He called it his ...",5,"[Analysis:, No, pain,, no, gain?, He, called, ...","[101, 4106, 1024, 2053, 3255, 1010, 2053, 5114..."
771,Saudi ministry to employ women Women will be ...,3,"[Saudi, ministry, to, employ, women, Women, wi...","[101, 8174, 3757, 2000, 12666, 2308, 2308, 209..."
772,The mayor of a small city receives suddenly th...,0,"[The, mayor, of, a, small, city, receives, sud...","[101, 1996, 3664, 1997, 1037, 2235, 2103, 8267..."
